# Meshtastic analysis experiments/reports 

To run:

* poetry install --with=analysis
* poe jyupyter

In [1]:
import meshtastic
import pyarrow.feather as feather

In [2]:
# sdir = '/home/kevinh/.local/share/meshtastic/slogs/20240626-152804'
sdir = '/home/kevinh/.local/share/meshtastic/slogs/latest'
dpwr = feather.read_table(f'{sdir}/power.feather').to_pandas()
# dslog = feather.read_table(f'{sdir}/slog.feather').to_pandas()

In [3]:
dpwr.head()

,time,average_mW,max_mW,min_mW
0,2024-06-26 16:41:54.128203,1.558931,1.987346,-0.000900
1,2024-06-26 16:41:54.333492,1.883926,2.000648,1.784251
2,2024-06-26 16:41:54.533642,1.886594,2.013956,1.784251
3,2024-06-26 16:41:54.738893,1.889409,2.009519,1.757879
4,2024-06-26 16:41:54.939203,1.890942,1.987346,1.779853


In [4]:
# dslog.head()

In [5]:
from dash import Dash, html, dcc, callback, Output, Input, dash_table
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

app = Dash()

def set_legend(f, name):
    f['data'][0]['showlegend']=True
    f['data'][0]['name']=name
    return f

df = dpwr
avg_pwr_lines = px.line(df, x='time', y='average_mW').update_traces(line_color='red')
set_legend(avg_pwr_lines, "avg power")
max_pwr_points = px.scatter(df, x='time', y='max_mW').update_traces(marker_color='blue')
set_legend(max_pwr_points, "max power")
min_pwr_points = px.scatter(df, x='time', y='min_mW').update_traces(marker_color='green')
set_legend(min_pwr_points, "min power")

# fig = avg_pwr_lines
# fig.add_trace(max_pwr_points)
fig = go.Figure(data=min_pwr_points.data + max_pwr_points.data + avg_pwr_lines.data)

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

# App layout
app.layout = [
    html.Div(children='Test Plottly with power consumption'),
    # dash_table.DataTable(data=df.to_dict('records'), page_size=10),
    dcc.Graph(figure=fig)
]


if __name__ == '__main__':
    app.run(debug=True)